In [1]:
!pip3 install apache-tvm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 8.6 MB/s eta 0:00:00


In [2]:
import torch
from torchvision import models

# 加载预训练的PyTorch模型
model = models.mobilenet_v2(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 46.5MB/s]


In [5]:
from torchvision import models, transforms
from PIL import Image

model.eval()

# 定义图像预处理流程
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# 加载并预处理图像
img = Image.open('cat.jpg')
input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0)  # 创建批次维度

# 检查是否有可用的GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
input_batch = input_batch.to(device)

# 进行前向传播
with torch.no_grad():
    output = model(input_batch)

# 获取预测结果
_, predicted = torch.max(output, 1)
print('预测类别索引:', predicted.item())

预测类别索引: 285


In [6]:
!pip install onnx onnxoptimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.1 kB 44.9 MB/s eta 0:00:00


In [7]:
import torch.onnx

# 导出模型到ONNX格式
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "mobilenetv2.onnx", verbose=True, export_params=True)

In [8]:
import tvm
print(tvm.__version__)
import onnx

# 加载ONNX模型
onnx_model = onnx.load("mobilenetv2.onnx")

0.11.1


In [9]:
import tvm
from tvm import relay

# 将ONNX模型转换为Relay表示
mod, params = relay.frontend.from_onnx(onnx_model)

In [18]:
# 选择目标平台
target = tvm.target.Target("llvm")  # 例如，为CPU编译

# 编译模型
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)
    lib.export_library('tvmTest.so')

In [11]:
import tvm.runtime
module = tvm.runtime.load_module('tvmTest.so')


In [17]:
# 创建执行模块
ctx = tvm.cpu(0)
from tvm.contrib import graph_executor
gmod = graph_executor.GraphModule(module['default'](tvm.device('cpu')))

# gmod.set_input("input.1", dummy_input.numpy())
gmod.set_input("input.1", input_batch.numpy())
gmod.run()

# 获取输出
output_data = gmod.get_output(0)

output_data = output_data.asnumpy()

import numpy as np
# 进行 softmax 以获取概率分布（可选）
probabilities = np.exp(output_data) / np.sum(np.exp(output_data), axis=1, keepdims=True)

# 获取预测类别索引
predicted_class = np.argmax(output_data, axis=1)

print('预测类别索引:', predicted_class[0])
# print(output_data)

预测类别索引: 285


In [19]:
!ls

cat.jpg  mobilenetv2.onnx  sample_data	tvmTest.so
